In [1]:
import pandas as pd
#from lxml import etree
import requests
from bs4 import BeautifulSoup

In [2]:
import json
import unicodedata
from datetime import datetime

---- using HTML response ----

In [3]:
import locale
locale.getdefaultlocale()

('sv_SE', 'UTF-8')

In [4]:
#locale.setlocale(locale.LC_TIME, "sv_SE.UTF-8")
#locale.setlocale(locale.LC_ALL, 'sv_SE')
locale.setlocale(locale.LC_TIME, "sv_SE")
#locale.setlocale(locale.LC_ALL, '')

'sv_SE'

In [5]:
url = "https://www.hemnet.se/salda/bostader"
params= {
    #'housing_form_groups':'apartments',
    'location_ids':'898472', #17853
    #'item_types':'bostadsratt',
    'by':'sale_date', #sort by sold date
    'order':'asc',
    'preferred_sorting':'true',
    'sold_age':'3m', #1d,4d,1w,1m and so on...
    'page':1
}
payload={}

headers = {
    'User-Agent': 'Mozilla/5.0'
}

response = requests.request("GET", url, headers=headers, data=payload, params=params)
soup = BeautifulSoup(response.content, "html.parser")
response.url

'https://www.hemnet.se/salda/bostader?location_ids=898472&by=sale_date&order=asc&preferred_sorting=true&sold_age=3m&page=1'

In [6]:
#list to add our objects to
item_list=[]

def read_pagedata(soupref):
    #find the ul with the results list
    result_ul=soupref.find('ul','sold-results')

    #get the list items with our data (filter out the ads)
    result_li=result_ul('li', 'sold-results__normal-hit')

    #print(result_li)

    #start loop...
    for item in result_li:
        #a dict to collect all key/values in
        item_dict={}

        #get tracking data json for ids...
        tracking_data=item.attrs['data-tracking-data']
        tracking_data_json=json.loads(tracking_data)

        #add ids
        item_dict['listingId']=tracking_data_json['listingId']
        item_dict['saleId']=tracking_data_json['saleId']
        
        #get the anchor href for the url
        result_li_a=item.find('a','sold-property-link')
        item_dict['url']=result_li_a['href']

        #get our sale listing info
        result_price_info=result_li_a.find('div','sold-property-listing__location')

        #address
        address=result_li_a.find('h2','sold-property-listing__heading').string.strip()
        #address=address.replace(' / ','/') #unicode getting in the way again?
        item_dict['address']=address

        #fee
        fee_el=result_li_a.find('div','sold-property-listing__fee')
        if fee_el:
            fee=fee_el.string.strip()
            fee=int("".join(unicodedata.normalize("NFKD",fee).strip('kr/mån').split()))
            item_dict['fee']=fee

        #living area
        #area=result_li_a.find('div','sold-property-listing__area').string.strip()
        #area=int("".join(unicodedata.normalize("NFKD",fee).strip('kr/mån').split()))
        #item_dict['area']=area
        area_el=result_li_a.find('div','sold-property-listing__area')
        if area_el:
            area=area_el.text.strip()
            item_dict["unprocessed"]=area
            area_arr="".join(area.split('m²')).split()
            
            # print(area_arr)

            span_el = result_li_a.find('span', 'listing-card__attribute--normal-weight')
            if span_el: 
                total_area=\
                    float(area_arr[0].replace(",", ".")) + \
                    float(area_arr[2].replace(",", "."))
                item_dict['living_area']=total_area
                if len(area_arr) > 4:
                    item_dict['rooms']=area_arr[3]
            else:
                item_dict['living_area']=area_arr[0]
                if len(area_arr) > 1:
                    item_dict['rooms']=area_arr[1] 
        
        #get our sale price info
        result_price_info=result_li_a.find('div','sold-property-listing__price-info')

        #ending date
        end_date_el=result_price_info.find('div',class_="sold-property-listing__sold-date")
        if end_date_el:
            end_date_str=end_date_el.string.strip()
            end_date_str=end_date_str.replace('Såld ','')
            end_date=datetime.strptime(end_date_str, '%d %B %Y')
            item_dict['end_date']=end_date

        #end price
        end_price_el=result_price_info.find('div',class_="sold-property-listing__subheading")
        if end_price_el:
            end_price_str=end_price_el.string.strip()
            end_price=int("".join(end_price_str.strip(' kr').replace('Slutpris ','').split()))
            item_dict['end_price']=end_price

        #end price/m2
        end_price_area_el=result_price_info.find('div',class_="sold-property-listing__price-per-m2")
        if end_price_area_el:
            end_price_area_str=end_price_area_el.string.strip()
            end_price_area=int("".join(unicodedata.normalize("NFKD",end_price_area_str).strip('kr/m2').split()))
            item_dict['end_price_area']=end_price_area
        
        #price change
        end_price_change_el=result_price_info.find('div',class_="sold-property-listing__price-change")
        if end_price_change_el:
            end_price_change_str=end_price_change_el.string.strip()
            end_price_change_str="".join(end_price_change_str.strip(' %').split())
            end_price_change=int(end_price_change_str[1:])
            end_price_change_sign=end_price_change_str[0]

            #calculate rate (percentage in float)
            if end_price_change !=0:
                item_dict['end_price_change_rate']=eval(f'100{end_price_change_sign}{end_price_change}')/100
            elif end_price_change==0:
                item_dict['end_price_change_rate']=100/100

            item_dict['end_price_change']=end_price_change
            item_dict['end_price_change_sign']=end_price_change_sign

        item_list.append(item_dict)

#print(item_list)

In [7]:
read_pagedata(soup)
next_link=soup.find('a','next_page')
while next_link:
    print(next_link['href'])
    next_response = requests.request("GET", "https://www.hemnet.se"+next_link['href'], headers=headers)
    next_soup = BeautifulSoup(next_response.content, "html.parser")
    read_pagedata(next_soup)
    next_link=next_soup.find('a','next_page')

/salda/bostader?by=sale_date&location_ids=898472&order=asc&page=2&preferred_sorting=true&sold_age=3m
/salda/bostader?by=sale_date&location_ids=898472&order=asc&page=3&preferred_sorting=true&sold_age=3m
/salda/bostader?by=sale_date&location_ids=898472&order=asc&page=4&preferred_sorting=true&sold_age=3m
/salda/bostader?by=sale_date&location_ids=898472&order=asc&page=5&preferred_sorting=true&sold_age=3m
/salda/bostader?by=sale_date&location_ids=898472&order=asc&page=6&preferred_sorting=true&sold_age=3m
/salda/bostader?by=sale_date&location_ids=898472&order=asc&page=7&preferred_sorting=true&sold_age=3m
/salda/bostader?by=sale_date&location_ids=898472&order=asc&page=8&preferred_sorting=true&sold_age=3m
/salda/bostader?by=sale_date&location_ids=898472&order=asc&page=9&preferred_sorting=true&sold_age=3m
/salda/bostader?by=sale_date&location_ids=898472&order=asc&page=10&preferred_sorting=true&sold_age=3m
/salda/bostader?by=sale_date&location_ids=898472&order=asc&page=11&preferred_sorting=true&

In [8]:
df=pd.json_normalize(item_list)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 691 entries, 0 to 690
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   listingId              691 non-null    int64         
 1   saleId                 691 non-null    int64         
 2   url                    691 non-null    object        
 3   address                691 non-null    object        
 4   fee                    690 non-null    float64       
 5   unprocessed            691 non-null    object        
 6   living_area            691 non-null    object        
 7   rooms                  691 non-null    object        
 8   end_date               691 non-null    datetime64[ns]
 9   end_price              691 non-null    int64         
 10  end_price_area         689 non-null    float64       
 11  end_price_change_rate  651 non-null    float64       
 12  end_price_change       651 non-null    float64       
 13  end_p

In [9]:
df

,listingId,saleId,url,address,fee,unprocessed,living_area,rooms,end_date,end_price,end_price_area,end_price_change_rate,end_price_change,end_price_change_sign
0,19090157,126117039812011944,https://www.hemnet.se/salda/lagenhet-2rum-sode...,"Bondegatan 50, 4 tr",1777.0,43 m²\n \n 2 rum,43,2,2022-10-01,3800000,88372.0,0.95,5.0,-
1,18829961,7210114487308123000,https://www.hemnet.se/salda/lagenhet-2rum-sode...,Repslagargatan 16,3361.0,45 m²\n \n 2 rum,45,2,2022-10-02,3800000,84444.0,0.96,4.0,-
2,18966690,4957632455148026499,"https://www.hemnet.se/salda/lagenhet-1,5rum-so...",Ringvägen 87A,2085.0,"43 m²\n \n 1,5 rum",43,"1,5",2022-10-02,3565000,82907.0,0.90,10.0,-
3,19010518,2869089503981924551,https://www.hemnet.se/salda/lagenhet-2rum-sode...,"Folkungagatan 63 , 3,5 tr, mot gård!",2618.0,51 m²\n \n 2 rum,51,2,2022-10-02,5750000,112745.0,1.05,5.0,+
4,19098459,2187763815569480668,"https://www.hemnet.se/salda/lagenhet-3,5rum-so...",Tideliusgatan 57,3024.0,"69 m²\n \n 3,5 rum",69,"3,5",2022-10-02,4900000,71014.0,1.01,1.0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686,19512321,7826102055536557072,https://www.hemnet.se/salda/lagenhet-2rum-sode...,Timmermansgatan 37,2585.0,45 m²\n \n 2 rum,45,2,2023-01-26,4360000,96889.0,1.09,9.0,+
687,19511045,9124860003505883870,https://www.hemnet.se/salda/lagenhet-4rum-kata...,"Vartoftagatan 9, 7 tr",4507.0,146 m²\n \n 4 rum,146,4,2023-01-26,16250000,111301.0,1.12,12.0,+
688,19505936,7650211919900293632,https://www.hemnet.se/salda/lagenhet-3rum-sode...,"Repslagargatan 23, vån 4",2969.0,77 m²\n \n 3 rum,77,3,2023-01-27,6870000,89221.0,1.10,10.0,+
689,19535502,2905477317003764455,https://www.hemnet.se/salda/lagenhet-1rum-sode...,Heleneborgsgatan 10B,2344.0,34 m²\n \n 1 rum,34,1,2023-01-27,3580000,105294.0,1.09,9.0,+


In [ ]:
df[df["fee"].isna()]

import json
import unicodedata
from datetime import datetime


#find the ul with the results list
result_ul=soup.find('ul','sold-results')

#get the list items with our data (filter out the ads)
result_li=result_ul('li', 'sold-results__normal-hit')

print(len(result_li))

#start loop...
for item in result_li:
    
    #get the anchor href for the url
    result_li_a=item.find('a','sold-property-link')

    #living area
    area=result_li_a.find('div','sold-property-listing__area').string.strip()
    area_arr="".join(area.split('m²')).split()
    rooms=area_arr[1]
    living_area=area_arr[0]
    #area=int("".join(unicodedata.normalize("NFKD",area).strip('kr/mån').split()))
    #address
    #address_str=result_price_info.find('div',class_='sold-property-listing__location').string.strip()
    #print(address_str)